In [1]:
import os
import shutil
import qiime2 as q2
import pandas as pd
import numpy as np
from biom import Table
from skbio import DistanceMatrix
from validation_util import set_root_path, decorate_path, load_run, remove_singletons
from qiime2.plugins.diversity.actions import core_metrics_phylogenetic, alpha_rarefaction
from qiime2.plugins.diversity.visualizers import beta_group_significance, adonis
from bp import parse_newick, write_newick
%matplotlib inline

In [2]:
# biomass can be either [high, low]
BP='150-bp'
BIOMASS='low'
DESCRIPTION=None
runs = [1,2,3]
# labs = ['kl']
labs = ['kl', 'core']
ROOT_PATH=set_root_path(BP, [BIOMASS, DESCRIPTION], runs, labs)

In [3]:
tables = []
metadatas = []
for run_i in range(len(runs)):
    for lab_j in range(len(labs)):
        run, lab = runs[run_i], labs[lab_j]
        table, metadata = load_run(BP, run, lab, BIOMASS, DESCRIPTION)
        tables.append(table)
        metadatas.append(metadata)
        if run_i == 0 and lab_j == 0:
            ids = metadata['original_id']
full_table, full_metadata= tables[0], metadatas[0]
full_table = full_table.merge(tables[1:])
print(full_table.shape)
full_table = remove_singletons(full_table)
full_table = full_table.remove_empty(axis='observation', inplace=False)
print(full_table.shape)
full_metadata = pd.concat(metadatas)
metadata_f_name = decorate_path('metadata.tsv', ROOT_PATH)

full_metadata.to_csv(metadata_f_name, sep='\t', index=True)
full_metadata.loc[full_metadata['original_id'].isin(ids)]
# full_metadata

(8568, 165) (4851, 165) 5016.666666666667
(15454, 175) (5947, 175) 19526.72
(12411, 172) (7219, 172) 8440.53488372093
(12484, 175) (5025, 175) 28245.257142857143
(9497, 172) (5685, 172) 6021.0058139534885
(19918, 175) (8007, 175) 25308.245714285713
(24044, 1034)
(19205, 1034)


,col,row,sex,diet,round,title,water,empo_1,empo_2,empo_3,...,physical_specimen_remaining,genomic_spike_in_copy_number,genomic_spike_in_description,metagenomic_spike_in_replicate,metagenomic_spike_in_copy_number,metagenomic_spike_in_amount_ng_per_ul,qiita_study_id,original_id,run,lab
sample_name,,,,,,,,,,,,,,,,,,,,,
run-1-kl14332.361162779,11,C,not applicable,not applicable,1_2,Matrix_pipeline_validation,not applicable,Free-living,Non-saline,Surface (non-saline),...,False,not applicable,syndna plasmids,1,1.61e+07,0.02,14332,14332.361162779,r1,kl
run-1-kl14332.361162780,11,A,not applicable,not applicable,1_2,Matrix_pipeline_validation,not applicable,Free-living,Non-saline,Surface (non-saline),...,False,2.76e+06,syndna plasmids,not applicable,not applicable,not applicable,14332,14332.361162780,r1,kl
run-1-kl14332.361162792,8,G,female,not applicable,1_2,Matrix_pipeline_validation,not applicable,Host-associated,Animal,Animal surface,...,False,not applicable,not applicable,not applicable,not applicable,not applicable,14332,14332.361162792,r1,kl
run-1-kl14332.361162794,12,O,not applicable,not applicable,1_2,Matrix_pipeline_validation,not applicable,Free-living,Non-saline,Surface (non-saline),...,False,not applicable,syndna plasmids,2,1.61e+07,0.02,14332,14332.361162794,r1,kl
run-1-kl14332.361162797,9,M,not applicable,not applicable,1_2,Matrix_pipeline_validation,not applicable,Free-living,Non-saline,Surface (non-saline),...,False,not applicable,syndna plasmids,1,2.59e+07,0.04,14332,14332.361162797,r1,kl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
run-3-core14332.361164197,10,B,not applicable,not applicable,1_2,Matrix_pipeline_validation,not applicable,Free-living,Non-saline,Surface (non-saline),...,False,5.51e+06,syndna plasmids,not applicable,not applicable,not applicable,14332,14332.361164197,r3,core
run-3-core14332.363197873,6,N,female,not applicable,1_2,Matrix_pipeline_validation,not applicable,Host-associated,Animal,Animal surface,...,False,2.21e+07,syndna plasmids,not applicable,not applicable,not applicable,14332,14332.363197873,r3,core
run-3-core14332.CONTROL.2.2.2,5,C,not applicable,not applicable,1_2,Matrix_pipeline_validation,not applicable,Control,Positive,Single strain,...,True,not applicable,not applicable,not applicable,not applicable,not applicable,14332,14332.CONTROL.2.2.2,r3,core


In [4]:
tree_path=os.path.join(BP, '{BP}.tre'.format(BP=BP))
tree = parse_newick(open(tree_path).read())
tree.shear(set(full_table.ids(axis='observation')))
tree_f_name = decorate_path('tree.tre', ROOT_PATH)
q2_tree_f_name = decorate_path('tree.qza', ROOT_PATH)

with open(tree_f_name, 'w') as f:
    write_newick(tree, f, include_edge=True)
    
! qiime tools import \
--type 'Phylogeny[Rooted]' \
--input-path {tree_f_name} \
--output-path {q2_tree_f_name} 

Imported 150-bp/low-runs-1-2-3-labs-kl-core/tree.tre as NewickDirectoryFormat to 150-bp/low-runs-1-2-3-labs-kl-core/tree.qza


In [5]:
q2_full_table = q2.Artifact.import_data(type='FeatureTable[Frequency]', view=full_table)
q2_full_metadata = q2.Metadata.load(metadata_f_name)
q2_full_tree = q2.Artifact.load(q2_tree_f_name)

In [6]:
rf_depths = [1500]

In [ ]:
#core diversity metrics @ different sequencing depths
for rf_depth in rf_depths:
    core_results = core_metrics_phylogenetic(q2_full_table,
                                             q2_full_tree,
                                             rf_depth,
                                             q2_full_metadata)
    
    results_dir_path = decorate_path(
        'core-metrics-%i'%rf_depth, ROOT_PATH
    )
    if os.path.isdir(results_dir_path):
        shutil.rmtree(results_dir_path)
    os.mkdir(results_dir_path)
    
    for name, q2art in zip(core_results._fields, core_results):
        out_ = os.path.join(results_dir_path,name)
        q2art.save(out_)
    rare_table_path = '{results_path}/rarefied_table.qza'.format(results_path=results_dir_path)
    table = q2.Artifact.load(rare_table_path).view(Table)
    print(table.shape)
    table = table.remove_empty(axis='observation', inplace=False)
    print(table.shape)
    q2_table = q2.Artifact.import_data(type='FeatureTable[Frequency]', view=table)
    q2_table.save(rare_table_path)

In [ ]:
# compare between labs
factors=['storage_solution','extraction_protocol']
major_factors=['sample_type', 'host_subject_id']
states=['lab', 'run']
# states=['lab']

lab_formula='"{major} + {factors} + {lab} +({i_major} + {i_factors}):{i_lab} "'.format(
    major='*'.join(major_factors),
    i_major='+'.join(major_factors),
    lab='*'.join(states),
    i_lab='*'.join(states),
    factors='*'.join(factors),
    i_factors='+'.join(factors)
)
#     distance_metrics = ['bray_curtis', 'jaccard', 'unweighted_unifrac', 'weighted_unifrac']
for rf_depth in rf_depths:
    results_dir_path = decorate_path(
        'core-metrics-%i'%rf_depth, ROOT_PATH
    )
        
#     distance_metrics = ['bray_curtis', 'jaccard', 'unweighted_unifrac', 'weighted_unifrac']
    distance_metrics = ['unweighted_unifrac']
    for d_metric in distance_metrics:
        d_matrix = os.path.join(
            results_dir_path, 
            '{metric}_distance_matrix.qza'.format(metric=d_metric))
        !qiime diversity adonis \
        --i-distance-matrix {d_matrix} \
        --m-metadata-file {metadata_f_name} \
        --p-formula {lab_formula} \
        --o-visualization {results_dir_path}/adonis-run-{d_metric}.qzv
